In [1]:
import requests
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd
import re

from geopy.geocoders import Nominatim
from IPython.display import Image
from IPython.core.display import HTML

from pandas.io.json import json_normalize
import folium

import time

## Setup FourSquare API


In [2]:
CLIENT_ID = 'AM4LR5YZCRHYW4CLN1L0QQZLF014UPH2KI2E3L55DVDTRN2D'
CLIENT_SECRET = 'RRPFF2VTPZI3PJ0EZFLWB3LYKQA02VQ5MUZHFEN2QLI05NOL'
ACCESS_TOKEN = ''
VERSION = '20210504'
LIMIT = 50
CODE = 'M101U03NXVBA4WWD3ZEIM4CMSV4CQ3FERO22212ESYX0N5XH#_=_'
ACCESS_TOKEN = 'TKPU2YRS21T0FAB0B4551QNQ4YXKQCKSWWKVD4412RDW224T'

## Get data from FourSquare API

### Write a method to get data
FourSquare API only allows to retrieve 50 most popular results per query. In order to retrieve all the stores across Toronto, we search by NEIGHBORHOOD using the coordinates obtained in previous assignment.

In [3]:
def foursquare(lat, lnt, search_query):
    # set up the query params
    radius = 30000 # set the radius to 30km
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lnt, ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)

    # request
    rsp = requests.get(url)
    rsp_code = rsp.status_code
    print('{} at {}, {} response status code: {}'.format(search_query, lat, lnt, rsp_code))
    
    # extract results
    if rsp_code == 200:
        js = rsp.json()
        venues = js['response']['venues']
        df = json_normalize(venues)
    
        return df


In [4]:
# get borough coors
hoods = pd.read_csv('./trt_boroughs.csv')[['Neighborhood', 'Latitude', 'Longitude']]
coors = zip(hoods.Latitude.values, hoods.Longitude.values)

In [ ]:
# search Tim Hortons in all neighborhoods
th = pd.DataFrame()

for lat, lnt in coors:
    try:
        result = foursquare(lat, lnt, 'Tim Hortons')
        th = pd.concat([th, result], ignore_index=True)
        
    except Exception as e:
        print(e)
        time.sleep(5)
        try:
            result = foursquare(lat, lnt, 'Tim Hortons')
            th = pd.concat([th, result], ignore_index=True)
        except Exception as e:
            print('Another error: ' + e)
            continue

th.to_csv('./TimHortons.csv', index=False)

Tim Hortons at 43.7532586, -79.3296565 response status code: 200
Tim Hortons at 43.7258823, -79.31557159999998 response status code: 200
Tim Hortons at 43.6542599, -79.3606359 response status code: 200
Tim Hortons at 43.718518, -79.46476329999999 response status code: 200
Tim Hortons at 43.6623015, -79.3894938 response status code: 200
Tim Hortons at 43.6678556, -79.53224240000002 response status code: 200
Tim Hortons at 43.8066863, -79.19435340000003 response status code: 500
Tim Hortons at 43.7459058, -79.352188 response status code: 200
Tim Hortons at 43.7063972, -79.309937 response status code: 200
Tim Hortons at 43.6571618, -79.37893709999999 response status code: 200
Tim Hortons at 43.709577, -79.44507259999997 response status code: 200
Tim Hortons at 43.6509432, -79.55472440000001 response status code: 200
Tim Hortons at 43.7845351, -79.16049709999999 response status code: 200
Tim Hortons at 43.72589970000001, -79.340923 response status code: 200
Tim Hortons at 43.69534390000001

In [ ]:
sc = pd.DataFrame()
for lat, lnt in coors:
    try:
        result = foursquare(lat, lnt, 'Second Cup')
        sc = pd.concat([sc, result], ignore_index=True)
        
    except Exception as e:
        print(e)
        time.sleep(5)
        try:
            result = foursquare(lat, lnt, 'Second Cup')
            sc = pd.concat([sc, result], ignore_index=True)
        except Exception as e:
            print('Another error: ' + e)
            continue

In [ ]:
# amend missing values
sc1 = foursquare(43.6678556, -79.53224240000002, 'Second Cup')
sc2 = foursquare(43.68641229999999, -79.4000493, 'Second Cup')
sc = pd.concat([sc, sc1, sc2], ignore_index=True)
sc.to_csv('./SecondCup.csv', index=False)

### Clean data
Remove duplicates in the dfs

In [ ]:
# define the 6 boroughs of Greater Toronto area
boroughs = ['Etobicoke', 'North York', 'Scarborough', 'York', 'East York', 'Toronto']

# import tim hortons results
# th = pd.read_csv('./TimHortons.csv')

# clean tim hortons results
th = th[th['location.city'].isin(boroughs)] # select stores in greater toronto
th.drop_duplicates(subset='id', inplace=True) # remove duplicates
print(th.shape)
th.head()

In [ ]:
# clean second cup results
sc = sc[sc['location.city'].isin(boroughs)]
sc.drop_duplicates(subset='id', inplace=True)
print(sc.shape)
sc.head()

### Draw map

In [ ]:
# create map object
mymap = folium.Map(location=[43.7181557, -79.5181405], zoom_start=11)


# draw tim hortons first in red
th_lats = th.iloc[:, 11].values
th_lnts = th.iloc[:, 12].values
for th_lat, th_lnt in zip(th_lats, th_lnts):
    folium.CircleMarker([th_lat, th_lnt],
                       radius=5,
                       color='red').add_to(mymap)
    
# draw second cups in blue
sc_lats = sc.iloc[:, 11].values
sc_lnts = sc.iloc[:, 12].values
for sc_lat, sc_lnt in zip(sc_lats, sc_lnts):
    folium.CircleMarker([sc_lat, sc_lnt],
                       radius=5,
                       color='blue').add_to(mymap)
    
mymap